In [1]:
# Read in raw data from webscaping

import numpy as np
import pandas as pd

zoocasa_data = pd.read_csv("#########/zoocasa original data.csv")
zoocasa_data.head()

,URL,status,Type,level,mgmt_fee,property_tax,Locker,Ensuite_Laundry,Balcony,Laundry_Level,...,Elevator,city,Province,Address,num_bed,num_bath,num_parking,sf_range,list_price,sold_price
0,https://www.zoocasa.com/toronto-on-real-estate...,Sold\n,Condo Apt,Loft,$763,"$3,706",None,Yes,None,NaN,...,NaN,"Toronto,",Ontario,110 - 27 Brock Ave,2 beds,1 baths,no parking,1000–1199 sq. ft.,"$749,900","756,000"
1,https://www.zoocasa.com/guelph-on-real-estate/...,Sold\n,Condo Townhouse,Stacked Townhse,$280,"$3,795",None,No,Open,Main,...,NaN,"Guelph,",Ontario,39 Kay Cres,2 beds,3 baths,2 parking,1600–1799 sq. ft.,"$539,900","540,000"
2,https://www.zoocasa.com/richmond-hill-on-real-...,Sold\n,Condo Apt,Apartment,$521,"$1,862",Owned,Yes,Open,Main,...,NaN,"Richmond Hill,",Ontario,1007 - 65 Oneida Cres,1 + 1 beds,1 baths,1 parking,600–699 sq. ft.,"$529,900","522,500"
3,https://www.zoocasa.com/hamilton-on-real-estat...,Sold\n,Condo Apt,Apartment,$362,$1,Owned,Yes,Open,Main,...,NaN,"Hamilton,",Ontario,233 - 125 Shoreview Pl,1 + 1 beds,1 baths,1 parking,600–699 sq. ft.,"$369,999","361,000"
4,https://www.zoocasa.com/clarington-on-real-est...,Sold\n,Condo Apt,Apartment,$343,NaN,Owned,Yes,Open,NaN,...,NaN,"Clarington,",Ontario,309 - 70 Shipway Ave,1 + 1 beds,2 baths,1 parking,700–799 sq. ft.,"$449,900","436,000"


In [ ]:
# Remove invalid records
i = zoocasa_data[zoocasa_data['num_bed'].isnull()].index
zoocasa_data.drop(i , inplace=True)

In [ ]:
# Data cleaning
zoocasa_data['mgmt_fee'] = zoocasa_data['mgmt_fee'].astype('str').str.replace(',', '')
zoocasa_data['mgmt_fee'] = zoocasa_data['mgmt_fee'].str.replace('$', '')
zoocasa_data['mgmt_fee'] = zoocasa_data['mgmt_fee'].astype('float')

zoocasa_data['property_tax'] = zoocasa_data['property_tax'].astype('str')
zoocasa_data['property_tax'] = zoocasa_data['property_tax'].str.replace(',', '')
zoocasa_data['property_tax'] = zoocasa_data['property_tax'].str.replace('$', '')
zoocasa_data['property_tax'] = zoocasa_data['property_tax'].astype('float')

zoocasa_data['num_bed'] = zoocasa_data['num_bed'].str.replace(' beds', '')
zoocasa_data['num_bed_temp'] = zoocasa_data['num_bed'].str.split('+')
zoocasa_data['num_bed_main'] = zoocasa_data['num_bed_temp'].apply(lambda x:x[0])
zoocasa_data['num_bed_main'] = zoocasa_data['num_bed_main'].astype('int')

def bedroom_secondary(bedroom_list):
    if len(bedroom_list) > 1:
        return bedroom_list[1]
    else: 
        return 0    
zoocasa_data['num_bed_secondary'] = zoocasa_data['num_bed_temp'].apply(bedroom_secondary)
zoocasa_data['num_bed_secondary'] = zoocasa_data['num_bed_secondary'].astype('int')

zoocasa_data['num_bath'] = zoocasa_data['num_bath'].str.replace(' baths', '')
zoocasa_data['num_bath'] = zoocasa_data['num_bath'].astype('int')

zoocasa_data['num_parking'] = zoocasa_data['num_parking'].str.replace(' parking', '')
zoocasa_data['num_parking'].replace(['no'], '0', inplace=True)

zoocasa_data['num_parking'] = zoocasa_data['num_parking'].astype('float')

zoocasa_data['sf_range'] = zoocasa_data['sf_range'].str.replace(' sq. ft.', '')
zoocasa_data['sf_range_temp'] = zoocasa_data['sf_range'].str.split('–')

def sf_range_to_number(sf_list):
    if len(sf_list) > 1:
        return (int(sf_list[0]) + int(sf_list[1])/2)
    else: 
        return 400  
zoocasa_data['sf'] = zoocasa_data['sf_range_temp'].apply(sf_range_to_number)
zoocasa_data['sf'] = zoocasa_data['sf'].astype('float')

zoocasa_data['list_price'] = zoocasa_data['list_price'].str.replace('$', '')
zoocasa_data['list_price'] = zoocasa_data['list_price'].str.replace(',', '')
zoocasa_data['list_price'] = zoocasa_data['list_price'].astype('float')

zoocasa_data['sold_price'] = zoocasa_data['sold_price'].str.replace('$', '')
zoocasa_data['sold_price'] = zoocasa_data['sold_price'].str.replace(',', '')
zoocasa_data['sold_price'] = zoocasa_data['sold_price'].astype('float')

In [ ]:
#Obtain geo info from Geopy library

import geopy
from geopy.geocoders import Nominatim

locator = Nominatim(user_agent="myGeocoder")

def geocoding(address):
    try: 
        location = locator.geocode(address, addressdetails=True)
        return location.raw
    except: 
        return None
    
zoocasa_data['geoinfo']=zoocasa_data['full_address'].apply(geocoding)

In [ ]:
#Extract geo info and create geo columns 
zoocasa_data.loc[~zoocasa_data['geoinfo'].isnull(), 'geoinfo'] = \
zoocasa_data.loc[~zoocasa_data['geoinfo'].isnull(), 'geoinfo'].apply(eval)

def extract_lon(dict_str):
    try:
        return dict_str['lon']
    except:
        return None
zoocasa_data.loc[zoocasa_data['geoinfo'].notnull(), 'lon'] = \
zoocasa_data.loc[~zoocasa_data['geoinfo'].notnull(), 'geoinfo'].apply(extract_lon)

def extract_lat(dict_str):
    try:
        return dict_str['lat']
    except:
        return None
zoocasa_data.loc[zoocasa_data['geoinfo'].notnull(), 'lat'] = \
zoocasa_data.loc[~zoocasa_data['geoinfo'].notnull(), 'geoinfo'].apply(extract_lat)

def extract_neighbourhood(geo_dict):
    try: 
        if 'neighbourhood' in geo_dict['address'].keys():
            return geo_dict['address']['neighbourhood']
        elif 'hamlet' in geo_dict['address'].keys():
            return geo_dict['address']['hamlet']
        elif 'quarter' in geo_dict['address'].keys():
            return geo_dict['address']['quarter']
        elif 'suburb' in geo_dict['address'].keys():
            return geo_dict['address']['suburb']
        else:
            return None
    except:
        return None  
zoocasa_data.loc[zoocasa_data['geoinfo'].notnull(), 'lat'] = \
zoocasa_data.loc[~zoocasa_data['geoinfo'].notnull(), 'geoinfo'].apply(extract_lat)

In [ ]:
#Obtain geo info from Google Map API (Access key needed), for records Geopy library failed to provide geocode

import googlemaps
from datetime import datetime

gmaps = googlemaps.Client(key='##########################')


def get_geoinfo_gmap(string):
    full_address = gmaps.geocode(string)
    return full_address

zoocasa_data.loc[~zoocasa_data['geoinfotemp'].isnull(), 'geoinfotemp'] = \
zoocasa_data.loc[~zoocasa_data['geoinfotemp'].isnull(), 'geoinfotemp'].apply(eval)

zoocasa_data['geoinfotemp'] = ''
zoocasa_data.loc[zoocasa_data['geoinfo'].isnull(), 'geoinfotemp'] = \
zoocasa_data.loc[zoocasa_data['geoinfo'].isnull(), 'full_address'].apply(get_geoinfo_gmap)



def extract_lat_g(g_list):
    try:
        return g_list[0]['geometry']['location']['lat']
    except:
        return None
    
zoocasa_data.loc[zoocasa_data['geoinfo'].isnull(), 'lat'] = \
zoocasa_data.loc[zoocasa_data['geoinfo'].isnull(), 'geoinfotemp'].apply(extract_lat_g)

def extract_lon_g(g_list):
    try:
        return g_list[0]['geometry']['location']['lng']
    except:
        return None
    
zoocasa_data.loc[zoocasa_data['geoinfo'].isnull(), 'lon'] = \
zoocasa_data.loc[zoocasa_data['geoinfo'].isnull(), 'geoinfotemp'].apply(extract_lon_g)


In [ ]:
#Export File for PBI Visulaization
zoocasa_data.to_csv('#######/zoocasa data with geo coding for PBI.csv')